# __Welcome to the Data Visualization Workshops!__

__Ayudante__: Gabriel Burgos S.

## __I. Introducción al Trabajo en los Workshops__

### __1.1. Data Visualization y Data Analysis__

1. __Importancia del manejo de Datos.__

2. __En qué ocupamos datos?__
- Optimización de procesos
- Análisis de mercado
- Control de calidad
- Etc.

### __1.2. Python__

Durante los workshop deberemos trabajar con _Python_ y algunas bibliotecas.

#### i- Qué es Python?
- Lenguaje de programación de alto nivel y orientado a objetos.
- Destacado por su simpleza.
- De código abierto extensible en C y C++.
- Se suele utilizar en aplicaciones de _Machine Learning_, _Ciencia de Datos_, ___Análisis de datos___, entre otros.


In [ ]:
print('Hola Mundo!')

In [ ]:
n = int(input())
for i in range(n):
    print('Lamamadelamama',end='')

#### ii- Por qué Python?
- De muy simple uso.
- Las bibliotecas para manejo de datos permiten mucha más cantidad de datos de manera rápida y automatizada.
- Utilizaremos '_jupyter notebooks_' los cuales son una manera intuitiva de revisar los códigos.


#### iii- Instalación:
Utilizar Python se puede lograr de diversas maneras, pero yo recomiendo dos:
1. [Visual Studio Code](https://code.visualstudio.com/) con extensiones de _Python_.
2. [Anaconda Navigator](https://www.anaconda.com/).

En este curso utilizaremos principalmente archivos `.ipynb`, por lo que lo importante es trabajar con _jupyter_.

- Bibliotecas:
	- [Pandas](https://pandas.pydata.org/)
	- [Polars](https://pola.rs/)
	- [Glob](https://docs.python.org/3/library/glob.html)
	- [Matplotlib](https://matplotlib.org/)
	- [Plotly](https://plotly.com/python/)

iv. Material Recomendado:
- Revisar el Starter Pack que les subiré:
	- Conceptos de Python básico y avanzado.
	- Guía completa de métodos de Pandas.
- Documentación Oficial:
	- [El Tutorial de Python Oficial](https://docs.python.org/es/3.13/tutorial/index.html)
	- [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html)
	- [Polars User Guide](https://docs.pola.rs/)
- Tutoriales Recomendados:
	- [Tutorial de Youtube sobre Python Básico](https://www.youtube.com/watch?v=D2cwvpJSBX4).
	- [Pandas en 10 minutos](https://www.youtube.com/watch?v=iGFdh6_FePU)

## II- Resolución de Workshop 1

Egresos hospitalarios por comuna de residencia. Conjunto de datos: https://deis.minsal.cl/#datosabiertos

##### **1. Descargue los egresos hospitalarios para el año 2020 y abra el archivo usando Pandas y Polars. ¿Qué diferencia a ambas bibliotecas?**

In [ ]:
#pip install pandas polars

In [ ]:
filepath = 'data/egresos hospitalarios/EGRESOS_2020/EGRE_DATOS_ABIERTOS_2020.csv'

__Pandas:__

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(filepath,
            encoding='latin-1',
			sep=';',
            )

__Polars:__

In [ ]:
import polars as pl

In [ ]:
pl.read_csv(filepath,
			encoding='latin-1',
			separator=';',
			null_values='*',
			)

##### **2. Descargue los egresos hospitalarios del año 2001 al 2020 y cree un único DataFrame usando Pandas. Pista: Use la biblioteca glob . Use este conjunto de datos para todos los análisis solicitados.**

In [ ]:
from glob import glob # Biblioteca para buscar archivos

In [ ]:
csv_list = glob('data/egresos hospitalarios/*/*.csv')
csv_list

In [ ]:
def read_csv_custom_pd(csv):
    return pd.read_csv(csv,
                         sep=';',
                         encoding='latin-1',
                         na_values='*',
                         )

def read_csv_custom_pl(csv):
    return pl.read_csv(csv,
						encoding='latin-1',
						separator=';',
						null_values=['*','NULL'],
                        schema_overrides={'PROCED':pl.Int64}
						)

In [ ]:
# PANDAS
total = len(csv_list) # para contador

print('Comenzando lectura...')

print(f'Leyendo {csv_list[-1].split('\\')[-1]} - {0/total:3.0%} ({1:2}/{total})')
df = read_csv_custom_pd(csv_list[-1])

for i,csv in enumerate(csv_list[-2::-1], start=2):
    print(f'Leyendo {csv.split('\\')[-1]} - {(i-1)/total:3.0%} ({i:2}/{total})')
    sub_df = read_csv_custom_pd(csv)
    df = pd.concat([df, sub_df])
    if i == 5:
        break
    
print('Lectura finalizada con éxito!')
df

# Duracion: 4m 46s

In [ ]:
# Correcion ESTABELCIMIENTO DE SALUD

# Rellenar valores nulos en 'Nombre' con los valores de 'Nombr'
df['PERTENENCIA_ESTABLECIMIENTO_SALUD'] = df['PERTENENCIA_ESTABLECIMIENTO_SALUD'].fillna(df['PERTENENCIA_ESTABLECIMIENTO_SALU'])

# Eliminar la columna 'Nombr' si ya no la necesitas
df.drop(columns=['PERTENENCIA_ESTABLECIMIENTO_SALU'], inplace=True)

# Verificar el resultado
df.head()

In [ ]:
# POLARS
total = len(csv_list) # para contador

print('Comenzando lectura...')

print(f'Leyendo {csv_list[0].split('\\')[-1]} - {0/total:3.0%} ({1:2}/{total})')
df = read_csv_custom_pl(csv_list[0])

for i,csv in enumerate(csv_list[1:], start=2):
    print(f'Leyendo {csv.split('\\')[-1]} - {(i-1)/total:3.0%} ({i:2}/{total})')
    sub_df = read_csv_custom_pl(csv)
    df = pl.concat([df, sub_df], how='vertical_relaxed')
    
print('Lectura finalizada con éxito!')
df

# Duracion: aprox. 40s (NUNCA ME FUNCIONOO)

##### **3. Determine el total de ﬁlas y columnas.**

In [ ]:
df.shape

In [ ]:
print(f'El dataset contiene {df.shape[0]:,} entradas (filas) y {df.shape[1]} variables (columnas).')

##### **4. Determine el total de ﬁlas, según sexo biológico.**

In [ ]:
df['SEXO'].value_counts()

In [ ]:
print(f'Hay {df['SEXO'].value_counts()['MUJER']:,} mujeres y {df['SEXO'].value_counts()['HOMBRE']:,} hombres.')

##### **5. Determine la patología que es más frecuente en el conjunto de datos.**

In [ ]:
df['DIAG1'].value_counts().sort_values(ascending=False).head()

In [ ]:
serie = df['DIAG1'].value_counts().sort_values(ascending=False)

print(f'El diagnóstico más frecuente es {serie.index[0]} con {serie.iloc[0]:,} casos.')

##### **6. Filtre únicamente los registros correspondientes a la comuna de Concepción.**

In [ ]:
comunas = df['GLOSA_COMUNA_RESIDENCIA'].dropna().unique()
comunas.sort()
comunas

In [ ]:
df[df['GLOSA_COMUNA_RESIDENCIA'] == 'Concepción']

##### **7. Calcule el total de ﬁlas correspondientes a las comunas de Cañete, Los Ángeles y Chillán.**

In [ ]:
df[df['GLOSA_COMUNA_RESIDENCIA'].isin(['Cañete', 'Los Ángeles', 'Chillán'])]

In [ ]:
len(df[df['GLOSA_COMUNA_RESIDENCIA'].isin(['Cañete', 'Los Ángeles', 'Chillán'])])

In [ ]:
print('Hay', len(df[df['GLOSA_COMUNA_RESIDENCIA'].isin(['Cañete', 'Los Ángeles', 'Chillán'])]), 'casos en las comunas de Cañete, Los Ángeles y Chillán.')

##### **8. Calcule el total de ﬁlas correspondientes a las comunas de Santa Juana, Maipú, Puente Alto y Pudahuel, en los años de egresos 2021 y 2022.**

In [ ]:
filtro_comuna = df['GLOSA_COMUNA_RESIDENCIA'].isin(['Santa Juana', 'Puente Alto', 'Maipú', 'Pudahuel'])
filtro_fecha = df['ANO_EGRESO']<=2019
filtro_fecha2 = df['ANO_EGRESO']>=2018

df_filtrado = df[filtro_comuna &
				 filtro_fecha &
				 filtro_fecha2
    			]

df_filtrado

In [ ]:
len(df_filtrado)

##### **9. Filtre los registros correspondientes a años de egreso pares. Evite deﬁnir año a año en el ﬁltro .**

In [ ]:
filtro_año_par = df['ANO_EGRESO'] % 2 == 0

df[filtro_año_par]

##### **10. Estime el porcentaje de egresos hospitalarios que se identiﬁca con una etnia en la región de Tarapacá, y compare dicho porcentaje con el correspondiente a la región de Los Ríos.**

In [ ]:
df['GLOSA_REGION_RESIDENCIA'].unique()

In [ ]:
df['ETNIA'].unique()

In [ ]:
filtro_tarapaca = df['GLOSA_REGION_RESIDENCIA'] == 'De Tarapacá'
filtro_losrios = df['GLOSA_REGION_RESIDENCIA'] == 'De Los Ríos'
filtro_etnia = df['ETNIA'] == 'Si se identifica con alguna etnia'

df_T = df[filtro_tarapaca]
df_LR = df[filtro_losrios]
df_T_et = df[filtro_tarapaca & filtro_etnia]
df_LR_et = df[filtro_losrios & filtro_etnia]

In [ ]:
total_tarapaca = len(df_T)
total_losrios = len(df_LR)
etnias_tarapaca = len(df_T_et)
etnias_losrios = len(df_LR_et)

print(f'La proporción de personas identificadas por etnias en Tarapacá es de {etnias_tarapaca/total_tarapaca:.3%} y de {etnias_losrios/total_losrios:.3%} en Los Ríos')

##### **11. ¿Cuál es la patología con mayor cantidad de egresos hospitalarios en la comuna de Concepción en 2022? ¿Fue la misma patología que en 2001?**

In [ ]:
filtro_conce = df['GLOSA_COMUNA_RESIDENCIA'] == 'Concepción'
filtro_2022 = df['ANO_EGRESO'] == 2020 # Reemplazo por 2020
filtro_2001 = df['ANO_EGRESO'] == 2016 # Reemplazo por 2016

In [ ]:
df_conce_2022 = df[filtro_conce & filtro_2022]
df_conce_2022['DIAG1'].value_counts().sort_values(ascending=False).head(1)

In [ ]:
df_conce_2001 = df[filtro_conce & filtro_2001]
df_conce_2001['DIAG1'].value_counts().sort_values(ascending=False).head(1)

##### **12. Calcule los días de estancia hospitalaria promedio, según diagnóstico.**

In [ ]:
df.groupby('DIAG1').agg({'DIAS_ESTADA':'mean'}).T

##### **13. ¿Qué porcentaje de los egresos hospitalarios presenta condición de egreso del paciente como fallecido?**

In [ ]:
df['CONDICION_EGRESO'].unique()

In [ ]:
filtro_fallecidos = df['CONDICION_EGRESO'] == 2

cantidad_fallecidos = len(df[filtro_fallecidos])
total = len(df)

print(f'El porcentaje de fallecidos es de {cantidad_fallecidos/total:.3%}')

##### **14. Compare las 10 patologías más frecuentes según el país de origen del paciente.**

In [ ]:
df['GLOSA_PAIS_ORIGEN'].unique()

In [ ]:
filtro_chileno = df['GLOSA_PAIS_ORIGEN'] == 'Chileno'
filtro_extranjero = df['GLOSA_PAIS_ORIGEN'] == 'extranjero'

top_diagnostico_chileno = df[filtro_chileno]['DIAG1'].value_counts().sort_values(ascending=False).head(10)
top_diagnostico_extranjero = df[filtro_extranjero]['DIAG1'].value_counts().sort_values(ascending=False).head(10)

datos = {'top chileno': top_diagnostico_chileno.index,'cantidad ch': top_diagnostico_chileno.values,'top extranjero': top_diagnostico_extranjero.index, 'cantidad ext':top_diagnostico_extranjero.values}

tabla_comparativa = pd.DataFrame(datos,index=[i for i in range(1,11)])
tabla_comparativa

##### **15. Estime la principal patología de cada comuna. ¿Identiﬁca algún patrón geográﬁco?**

In [ ]:
df.groupby('GLOSA_COMUNA_RESIDENCIA').agg()

##### **16. Calcule el total de egresos hospitalarios por año.**

In [ ]:
df['ANO_EGRESO'].value_counts()

##### **17. Calcule el total de egresos hospitalarios por año para la región de Biobío.**

In [ ]:
filtro_biobio = df['GLOSA_REGION_RESIDENCIA'] == 'Del Bíobío'

df[filtro_biobio]['ANO_EGRESO'].value_counts()

##### **18. ¿Cuáles son los porcentajes de egresos según previsión de salud?**

##### **19. Sin incluir comunas de la región metropolitana, ¿cuáles son las comunas con mayor mediana de días de estada hospitalaria?**

##### **20. Calcule el total de patologías según grupo de edad y sexo biológico.**

##### **21. Guarde los datos obtenidos en la pregunta 12 en un nuevo archivo llamado “egresos.xlsx”. No olvides usar reset_index() .**